In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [14]:
numger = 2;
numload = 1;
ger = [f'Gerador {i}' for i in range(1, numger + 1)]
custo_a = [1, 2]
custo_b = [4, 2]
custo_c = [5, 5]
carga = [f'Carga {i}' for i in range(1, numload + 1)]
demanda = 30

#Modelo Concreto
model = ConcreteModel()
model.ger = Set(initialize=ger)
model.carga = Set(initialize=carga)

# Parâmetros
model.a = Param(model.ger, initialize={ger[i]: custo_a[i] for i in range(numger)})
model.b = Param(model.ger, initialize={ger[i]: custo_b[i] for i in range(numger)})
model.c = Param(model.ger, initialize={ger[i]: custo_c[i] for i in range(numger)})
model.demanda = Param(model.carga, initialize={carga[i]: demanda for i in range(numload)})

#Variáveis
model.pot = Var(model.ger, domain=NonNegativeReals)

#Restrição
model.restr = Constraint(expr = sum(model.pot[i] for i in model.ger) - sum(model.demanda[i] for i in model.carga) == 0)

#FOB
def FOB(model):
    custo = sum(model.a[i]*model.pot[i]**2 + model.b[i]*model.pot[i] + model.c[i] for i in model.ger)
    return custo
model.FOB = Objective(rule=FOB, sense=minimize)

# Adicionar o componente para armazenar duais
model.dual = Suffix(direction=Suffix.IMPORT)

#Solver
solver = SolverFactory('ipopt')
solver.options['dual_inf_tol'] = 1e-6
results = solver.solve(model, tee=False)

In [15]:
#Resultados:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', results.solver.status, '\n')
print('Condição de Término:', results.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.FOB), '\n')

# Exibindo as variáveis de potência vendida de cada lance vendedor
print("Resultados das Potências Vendidas (Lances dos Vendedores):")
venda_total = 0
for gerador in model.ger:
    venda_total += model.a[gerador]*model.pot[gerador].value**2 + model.b[gerador]*model.pot[gerador].value + model.c[gerador]
    print(f"{gerador}: {model.pot[gerador].value} MWh")
    print(f"Preço do {gerador}: R${model.a[gerador]*model.pot[gerador].value**2 + model.b[gerador]*model.pot[gerador].value + model.c[gerador]}")
print(f'Venda total:{venda_total}$')

# Verificação e impressão do custo marginal
if model.restr in model.dual:
    custo_marginal = model.dual[model.restr]
    print("Custo Marginal: R$", custo_marginal)
else:
    print("Valor dual da restrição não está disponível.")

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 709.6666666666667 

Resultados das Potências Vendidas (Lances dos Vendedores):
Gerador 1: 19.666666666647487 MWh
Preço do Gerador 1: R$470.4444444436133
Gerador 2: 10.333333333352515 MWh
Preço do Gerador 2: R$239.22222222305342
Venda total:709.6666666666667$
Custo Marginal: R$ 43.33333333316758
